In [13]:
# Package import
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

In [14]:
# Data import and reading/assignation of values

file_path = Path('DataRefValue.xlsx')

data = pd.read_excel(file_path, sheet_name= 'Sheet1')

ref_values = data['RefValue']

value_columns = ['Value1', 'Value2', 'Value3', 'Value4', 'Value5', 'Value6'] 

#data = data.dropna(subset=value_columns, how='all')

#ref_values = data['RefValue']
time_periods = ['Duration1', 'Duration2', 'Duration3', 'Duration4', 'Duration5', 'Duration6']
#Evalue_columns = ['Value1', 'Value2', 'Value3', 'Value4', 'Value5', 'Value6'] 

valid_ref_values = data.loc[~data['Value1'].isna(), 'RefValue']

value_types = ['Emergency Response', 'Occupational', 'General Public']

data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

data.head(10)

,RefValue,Value1,Duration1,Value2,Duration2,Value3,Duration3,Value4,Duration4,Value5,Duration5,Value6,Duration6,Type,Shape,MarkColor,CatColor
0,AEGL-1,420.0,0.17,240.0,0.5,170.0,1.0,58.0,4.0,29.0,8.0,NaN,NaN,Emergency Response,Diamond,Red,Red
1,AEGL-2,6500.0,0.17,3600.0,0.5,2600.0,1.0,1300.0,4.0,650.0,8.0,NaN,NaN,Emergency Response,Diamond,Gold,Red
2,AEGL-3,31000.0,0.17,18000.0,0.5,13000.0,1.0,6500.0,4.0,3300.0,8.0,NaN,NaN,Emergency Response,Diamond,Blue,Red
3,ERPG-1,156.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Red,Red
4,ERPG-2,470.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Gold,Red
5,ERPG-3,3190.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Triangle,Blue,Red
6,PAC-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Red,Red
7,PAC-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Gold,Red
8,PAC-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emergency Response,Diamond,Blue,Red
9,ACGIH-Ceiling*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Occupational,Circle,Red,Orange


In [16]:
fig = go.Figure()

'''
# Colors for legend categories, added as CatColors
category_colors = {
    "Emergency Response" : 'Red',
    "Occupational" : 'Gold',
    "General Public" : 'Green',
}
'''
shape_mapping = {
    "Diamond": "diamond",
    "Triangle": "triangle-up",
    "Circle": "circle",
    "Square": "square",
    "Cross": "cross",
}

color_mapping = {
    "Red": "red",
    "Gold": "gold",
    "Blue": "blue",
    "Orange": "orange",
    "Yellow": "yellow",
    "Peach": "#FFDAB9",
    "Beige": "#F5F5DC",
    "White": "white",
    "Pink": "pink",
    "Light Green": "#90EE90",
    "Green": "green",
    "Pale Green": "#98FB98",
}

for idx, ref_value in enumerate(ref_values):
    durations = data.loc[idx, time_periods].dropna().values
    values = data.loc[idx, value_columns].dropna().values
    shape = data.loc[idx, 'Shape']
    color_name = data.loc[idx, 'MarkColor']

    marker_symbol = shape_mapping.get(shape, "circle")
    marker_color = color_mapping.get(color_name, "gray")

    showlegend = ref_value in valid_ref_values

    category = data.loc[idx, 'Type']

    if category == "Emergency Response":
        legend_title = "Emergency Response"
        legend_bgcolor = "Red"
    elif category == "Occupational":
        legend_title = "Occupational"
        legend_bgcolor = "Gold"
    else:
        legend_title = "General Public"
        legend_bgcolor = "Green"

    fig.add_trace(go.Scatter(
        x=durations,
        y=values,
        mode='lines+markers',
        name=ref_value,
        legendgroup=category,
        marker=dict(symbol=marker_symbol, color=marker_color)
    ))

annotations = []

legend_x = 1.05
legend_y = 1.0
legend_gap = 0.1

legend_y2 = 1.0
legend_gap2 = 0.1

fig.update_layout(
    title="Benzene Inhalation Reference Values",
    xaxis_title="Duration (Hours)",
    yaxis_title="Concentration mg/m3",
    xaxis_type="log",
    yaxis_type="log",
    template="plotly_white",
    height = 800,
    width = 1000,

    legend = dict(
        # groupclick = "toggleitem",
        tracegroupgap = 5,
        #orientation = "h",
        #x=0,
        #y=-0.2,
        #traceorder="normal",
        # bgcolor = "LightSteelBlue",
        font=dict(
            color = "Grey",
            size=8
        )
    ),
    annotations = annotations

)

# lines to add, specified by x-position
lines = {'24 Hours':24,'30 Days':720,'7 years':61320,'70 Years':613200,'Cancer Ranges':675000}

for k in lines.keys():
    #print(k)
    fig.add_shape(type='line',
                yref="y",
                xref="x",
                x0=lines[k],
                y0=0.00001,
                x1=lines[k],
                y1=150000,
                line=dict(color='black', width=1))


# TODO Add labels for lines 

for lines, props in lines.items():
    annotations.append(dict(
        #y = 1.0, x=props["x"],
        #xref = x, 
        #yref = 'paper',
        text = lines,
        showarrow = False, 
        font = dict(color="White", size =8)
    ))
    legend_y2 -= legend_gap2


fig.show()